In [1]:
from backend.src.extractors.data_ingestion import DataIngestion
from backend.src.extractors.image_summarizer import ImageSummarizer
from backend.src.extractors.text_summarizer import TextSummarizer
from backend.src.constants import *
from backend.src.config.configuration import ConfigurationManager
import os

from langchain_openai import ChatOpenAI

/Users/goldyrana/miniconda3/envs/bounce/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/goldyrana/mess/deep_learning/projects/rag/backend/notebooks


In [2]:
os.getcwd()

'/Users/goldyrana/mess/deep_learning/projects/rag/backend/notebooks'

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.chdir('../')

In [5]:
test_pdf_path = "data/raw_pdfs/2023_removed.pdf"
model = ChatOpenAI(temperature=0, model="gpt-4o")

config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
data_ingestion_config = config_manager.get_data_ingestion_params()
image_summarizer_config = config_manager.get_image_summarizer_params()
text_summarizer_config = config_manager.get_text_summarizer_params()

data_ingestion = DataIngestion(data_ingestion_config)
image_summarizer = ImageSummarizer(image_summarizer_config, model)

2025-02-10 20:12:17,610 - root - INFO - Yaml read successfully from ../config.yaml
2025-02-10 20:12:17,611 - root - INFO - Yaml read successfully from ../params.yaml
2025-02-10 20:12:17,611 - root - ERROR - Exception occured while reading yaml file from                         location: ../params.yaml
 First argument must be mapping or iterable
2025-02-10 20:12:17,613 - root - INFO - Json object read sucessfully 
2025-02-10 20:12:17,613 - root - INFO - Json object read sucessfully 


In [12]:
test_pdf_path = "data/raw_pdfs/2023_removed.pdf"
model = ChatOpenAI(temperature=0, model="gpt-4o")

config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
data_ingestion_config = config_manager.get_data_ingestion_params()
image_summarizer_config = config_manager.get_image_summarizer_params()
text_summarizer_config = config_manager.get_text_summarizer_params()

data_ingestion = DataIngestion(data_ingestion_config)
image_summarizer = ImageSummarizer(image_summarizer_config, model)

raw_pdf_elements, report_dir = data_ingestion.process_pdf(test_pdf_path,
                                                          save=True,
                                    strategy="hi_res", # 
                                    split_pdf_page=True,  # to process each page seprately
                                    split_pdf_allow_failed=True, # continue processing even if some pages fail
                                    extract_images_in_pdf=True,
                                    infer_table_structure=True,
                                    chunking_strategy="by_title",
                                    extract_image_block_types = ["Image" , "Table"],
                                    max_characters=4000,
                                    new_after_n_chars=3800,
                                    combine_text_under_n_chars=2000)


image_path = "data/reports/2023_removed/images"    
images_path = image_summarizer.get_image_path(image_path)
print(image_path)
encoded_images = []
image_summaries = []
meta_data = []

for image in images_path:
    encoded_image, image_source_path = image_summarizer.encode_image(image)
    image_summaries.append(image_summarizer.image_summarize(encoded_image))
    encoded_images.append(encoded_image)
    meta_data.append({"source": image_source_path,
                      "type": "image",
                      "title": image.split("/")[-1]})
    
image_summaries = image_summarizer.add_metadata(encoded_images, image_summaries, metadata=meta_data, automatic_metadata=False)

# # text summaries
text_summarizer = TextSummarizer(text_summarizer_config, model)
text_data = text_summarizer.get_text_data(raw_pdf_elements)

In [7]:
len(image_summaries), len(meta_data), len(encoded_images)

(37, 37, 37)

# vector store testing

In [14]:
from backend.src.storage.chroma_storage import VectorDatabase
from langchain_openai import OpenAIEmbeddings


vector_database_config = config_manager.get_vectordatabase_config()
vector_database = VectorDatabase(vector_database_config)
vectorstore = vector_database.init_chromadb(embeddings=OpenAIEmbeddings())
retriever = vector_database.get_retriever(vectorstore)
print(retriever)

2025-02-10 20:23:02,120 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-10 20:23:02,231 - chromadb.config - DEBUG - Starting component System
2025-02-10 20:23:02,232 - chromadb.config - DEBUG - Starting component Posthog
2025-02-10 20:23:02,232 - chromadb.config - DEBUG - Starting component OpenTelemetryClient
2025-02-10 20:23:02,232 - chromadb.config - DEBUG - Starting component SqliteDB
2025-02-10 20:23:02,238 - chromadb.config - DEBUG - Starting component SimpleQuotaEnforcer
2025-02-10 20:23:02,239 - chromadb.config - DEBUG - Starting component SimpleRateLimitEnforcer
2025-02-10 20:23:02,239 - chromadb.config - DEBUG - Starting component LocalSegmentManager
2025-02-10 20:23:02,240 - chromadb.config - DEBUG - Starting component LocalExecutor
2025-02-10 20:23:02,240 - chromadb.config - DEBUG - Starting component SegmentAPI
2025-02-10 20:23:02,245 - chromadb.api.segmen

vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x35e7f74c0> docstore=<langchain_core.stores.InMemoryStore object at 0x335278a30> search_kwargs={}


2025-02-10 20:23:02,750 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): us.i.posthog.com:443
2025-02-10 20:23:04,067 - urllib3.connectionpool - DEBUG - https://us.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15


# Store text data


In [16]:
raw_text, text_summaries, summary_metadata = text_summarizer.generate_summary(text_data)

2025-02-10 20:23:23,133 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Act as an assistant tasked with summarizing the tables and text. Give a concise summary of the table or text. Table or text chunk: Cautionary Statement\n\nThis presentation provides management\'s current operational plan for ConocoPhillips over roughly the next decade, for the assets currently in our portfolio, and is subject to multiple assumptions, including, unless otherwise specifically noted: * anoil price of $60/BBL West Texas Intermediate in 2022 dollars, escalating at 2.25% annually;\n\n* anoil price of $65/BBL Brent in 2022 dollars, escalating at 2.25% annually;\n\n* agas price of $3.75/MMBTU Henry Hub in 2022 dollars, escalating at 2.25%\n\nannually;\n\nan international gas price of $8/MMBTU Title Transfer Facility & Japan Korea Marker in 2022 dollars, escalating at 2.25% annually;\n\n* cost and capital 

In [24]:
text_summaries = text_summarizer.add_metadata(raw_text, text_summaries, summary_metadata)

In [25]:
vector_database.store_to_vb(text_summaries, retriever)

2025-02-10 20:25:00,088 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x3620ee040>, 'json_data': {'input': [[791, 15864, 50729, 1221, 13441, 92777, 3153, 6, 25605, 3197, 369, 279, 1828, 13515, 11, 3196, 389, 1510, 12032, 323, 3892, 32946, 13, 5422, 32946, 2997, 5707, 7729, 315, 400, 1399, 16675, 9574, 369, 4410, 8421, 61748, 323, 400, 2397, 16675, 9574, 369, 47431, 11, 2225, 77157, 520, 220, 17, 13, 914, 4, 30171, 11, 323, 6962, 7729, 315, 400, 18, 13, 2075, 38460, 18066, 52, 369, 18063, 27636, 323, 400, 23, 38460, 18066, 52, 369, 6625, 11987, 11, 1101, 77157, 520, 220, 17, 13, 914, 4, 30171, 13, 578, 3197, 32238, 2853, 323, 6864, 71389, 304, 1584, 449, 3430, 12992, 323, 5764, 12782, 13426, 304, 8515, 6530, 51165, 1405, 8581, 13, 578, 2246, 5727, 4741, 31348, 12518, 922, 3938, 4455, 11, 6787, 11, 323, 25605, 3135, 11, 11054, 555, 3230, 57726, 11, 323, 374, 3917, 311,

# Store Image data

In [15]:
vector_database.store_to_vb(image_summaries, retriever)

2025-02-10 20:23:16,685 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x325a87e50>, 'json_data': {'input': [[791, 2217, 374, 264, 4382, 6437, 4706, 389, 264, 4251, 4092, 13, 1102, 4519, 264, 31052, 323, 264, 2246, 13, 578, 31052, 374, 44894, 449, 264, 5950, 315, 12706, 11, 11383, 14393, 264, 5410, 31052, 6932, 13, 9479, 311, 433, 11, 1070, 374, 264, 2246, 449, 264, 18160, 1879, 520, 279, 1948, 11, 19392, 6020, 477, 24043, 2262, 13, 578, 2246, 1101, 706, 264, 2478, 16600, 5238, 11, 23377, 1495, 477, 12678, 13, 2684, 527, 912, 40099, 477, 3230, 5219, 304, 420, 2217, 13], [791, 2217, 374, 264, 42415, 3158, 9676, 94012, 279, 220, 605, 4771, 5788, 24549, 9974, 6650, 4478, 320, 34, 82256, 8, 369, 2204, 4907, 5788, 5528, 13, 578, 12414, 8183, 11105, 5788, 5990, 11, 449, 62700, 13160, 520, 220, 2636, 11, 220, 16, 11, 931, 11, 220, 16, 11, 2636, 11, 323, 220, 17, 11, 931, 382